<a href="https://colab.research.google.com/github/aminecloud9/PFE/blob/main/CICIDS2017_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Download dataset

In [2]:
data=pd.read_csv('/content/drive/MyDrive/globalData.csv')
#Labels=pd.read_csv('/content/drive/MyDrive/CICIDS2017_Labels.csv')

In [3]:
Y = data.values[:,-1]
X = data.iloc[: , :-1]

Data scaling

In [4]:
scaler = StandardScaler()
X= np.nan_to_num(X)
X = scaler.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:808: RuntimeWarning: invalid value encountered in true_divide
  X /= self.scale_


Split dateset

In [5]:
#Import Module
from sklearn.model_selection import train_test_split


train_X, test_X, train_y, test_y = train_test_split(X, Y, 
                                                     train_size=0.7,
                                                     test_size=0.3,
                                                     random_state=122)

In [9]:
del(X)
del(Y)

In [ ]:
train_X[:,pd.isna(train_X).any()].tolist()

In [27]:
# reshape input to be [samples, time steps, features]
import keras

#X_train = np.reshape(data.values[:,:-1].astype('float32'), (data.values[:,:-1].shape[0], 1, data.values[:,:-1].shape[1]))
X_train = np.reshape(train_X.astype('float32'), (train_X.shape[0], 1, train_X.shape[1]))
X_test = np.reshape(test_X.astype('float32'), (test_X.shape[0], 1, test_X.shape[1]))
#y_train = keras.utils.to_categorical(pd.factorize(data.values[:,77])[0],15)
test_y = pd.factorize(test_y)
train_y = pd.factorize(train_y)
y_train = keras.utils.to_categorical(train_y[0],train_y[1].shape[0])
y_train = y_train.astype('int')
y_train = np.reshape(y_train,(y_train.shape[0],1,y_train.shape[1]))
y_test = keras.utils.to_categorical(test_y[0],train_y[1].shape[0])
y_test = y_test.astype('int')
y_test = np.reshape(y_test,(y_test.shape[0],1,y_test.shape[1]))

In [ ]:
import time
data_dim = X_train.shape[2]
timesteps = 1
num_classes = y_train.shape[2]
batch_size = 1

model = Sequential()
model.add(LSTM(50, return_sequences=True, stateful=False, batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(Dropout(0.1))
model.add(LSTM(50, return_sequences=True, stateful=False))
model.add(Dropout(0.1))
model.add(LSTM(50, stateful=True))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


start = time.time()

start_time = time.time()
model.fit(X_train, y_train,batch_size=batch_size, epochs=2, shuffle=False)
finish_time = time.time() - start_time
#model.save("/content/drive/My Drive/final_model_NSL-KDD_brute_Multiclass.h5")

In [ ]:
import time
#train_object_num=len(train_data)
#print(test[0])
#model training
batch_size = 512
model=Sequential()
model.add(LSTM(32,input_dim=X_train.shape[2],return_sequences=True,kernel_initializer='uniform',activation='relu'))
model.add(LSTM(32,kernel_initializer='uniform',return_sequences=True,activation='relu'))
#model.add(LSTM(128,kernel_initializer='uniform',activation='relu'))
#model.add(Dense(256,activation='relu'))
#model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
#model.add(Dense(16,activation='relu'))
model.add(Dense(y_train.shape[2],activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='NADAM',metrics=['accuracy'])
model.summary()
start_time = time.time()
history = model.fit(X_train,y_train,validation_split=.3,epochs=100,batch_size=batch_size,verbose=1)
finish_time = time.time() - start_time
model.save("/content/drive/My Drive/model_CICIDS_brute_data_valsplit.3.h5")

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, None, 32)          13056     
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
dense_9 (Dense)              (None, None, 64)          2112      
_________________________________________________________________
dense_10 (Dense)             (None, None, 7)           455       
Total params: 23,943
Trainable params: 23,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2710/2710 [==============================] - 26s 8ms/step - loss: nan - accuracy: 0.7962 - val_loss: nan - val_accuracy: 0.8034
Epoch 2/100
2710/2710 [==============================] - 23s 8ms/step - loss: nan - accuracy: 0.8033 - val_l

In [30]:
batch_size= 1024
lrate = 0.1
model = Sequential()
model.add(Dense(X_train.shape[2], input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1024, input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.1))
model.add(Dense(768, input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.1))
model.add(Dense(512, input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.1))
model.add(Dense(256, input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.1))
model.add(Dense(128, input_dim=X_train.shape[2], activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.1))
model.add(Dense(y_train.shape[2], activation='softmax'))
	# compile model
opt = SGD(lr=lrate)
#loss = categorical_focal_loss([67343,45927,995,11656,52])
model.compile(loss= 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = model.fit(X_train, y_train, validation_split=.3,batch_size=batch_size, epochs=10, verbose=1)

Epoch 1/10
1355/1355 [==============================] - 268s 197ms/step - loss: nan - accuracy: 0.8034 - val_loss: nan - val_accuracy: 0.8034
Epoch 2/10
1355/1355 [==============================] - 265s 196ms/step - loss: nan - accuracy: 0.8027 - val_loss: nan - val_accuracy: 0.8034
Epoch 3/10
1355/1355 [==============================] - 264s 195ms/step - loss: nan - accuracy: 0.8022 - val_loss: nan - val_accuracy: 0.8034
Epoch 4/10
1355/1355 [==============================] - 271s 200ms/step - loss: nan - accuracy: 0.8026 - val_loss: nan - val_accuracy: 0.8034
Epoch 5/10
1355/1355 [==============================] - 264s 195ms/step - loss: nan - accuracy: 0.8032 - val_loss: nan - val_accuracy: 0.8034
Epoch 6/10
1355/1355 [==============================] - 265s 195ms/step - loss: nan - accuracy: 0.8030 - val_loss: nan - val_accuracy: 0.8034
Epoch 7/10
1355/1355 [==============================] - 265s 196ms/step - loss: nan - accuracy: 0.8030 - val_loss: nan - val_accuracy: 0.8034
Epoch 

In [ ]:
loss, accuracy = model.evaluate(X_test,y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)

print("execution time :  ",finish_time)

In [ ]:
#saving model
model.save("/content/drive/MyDrive/NSL-KDD_results_3LSTM_1Dense_20epoch_1batch_model.h5")

In [35]:
from keras.models import load_model
#model = load_model('/content/drive/MyDrive/NSL-KDD_results_SimpleRNN_model.hdf5')
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

#print("--- %s seconds ---",finish_time)
y_pred = model.predict_classes(X_train)
#np.savetxt('/content/drive/MyDrive/NSL-KDD_results_SimpleRNN_predicted.txt', np.transpose([y_test,y_pred]), fmt='%s')

26539/26539 [==============================] - 155s 6ms/step - loss: nan - accuracy: 0.0816

Loss: nan, Accuracy: 8.16%


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
